<a href="https://colab.research.google.com/github/mclabs74/inference_nbs/blob/main/yolov5/pretrained_pil_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

based on https://github.com/ultralytics/yolov5/issues/36

### install yolo5 requirements

In [ ]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

### Download model
Download the model, if it does not exist, and instantiate model object

In [ ]:
import cv2
import torch
from PIL import Image
from pathlib import Path

# folder for model folders. check to see if it exists, and if so
# don't download
model_path = Path("./model")
model_folder = Path(model_path/"ultralytics_yolov5_master")
if model_folder.is_dir() == False:
    torch.hub.set_dir(model_path)

# instantiate model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

### inference api decorators
Install inference api decorators. This is necessary for the inference servers to know your predict datatypes 

In [ ]:
!pip install -q git+https://github.com/matthewchung74/inference_params.git
from inference_params.inference_params import inference_test, FieldType, inference_predict

### Predict function
Define your predict function and decorate with inference wrapper and input/output types

In [ ]:
import torch.nn.functional as F
import torch
import io
import base64
import json


# text input will be the label for the app input
input = {"image input": FieldType.PIL}
# result will be the label for the app output
output = {"result image": FieldType.PIL, "result text": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    img = params['image input']
    results = model(img)  # inference
    results.imgs # array of original images (as np array) passed to model for inference
    results.render()  # updates results.imgs with boxes and labels
    pil_image = Image.fromarray(results.imgs[0])
    records = results.pandas().xyxy[0].to_csv()
    return {"result image": pil_image, "result text": records}

### Test it
Make sure you have your `in_colab` functions so your tests are not executed on every request

In [ ]:
from inference_params.inference_params import in_colab

if in_colab():
    torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/zidane.jpg', 'zidane.jpg')
    img = Image.open('zidane.jpg')  # PIL image

    from inference_params.inference_params import in_colab
    from matplotlib.pyplot import imshow
    import numpy as np

    from google.colab.patches import cv2_imshow
    result, duration = predict({'image input': img})
    result_image = result['result image']
    result_text = result['result text']

    print(result_text)
    display(result_image)

### Run Inference Test
This function `inference_test` saves your input and output types to disk, which are needed for inference to know your datatypes 

In [ ]:
from inference_params.inference_params import in_colab

if in_colab():
    inference_test(predict_func=predict, params={'image input': img})

### Clear outputs
You should clear your outputs so your notebook is not too large. Go to File->Clear Outputs

### Commit your file to github.
Then go to inference.codes to upload your file.